CSP - constraint satiscation problem
for Chord fingering

In [1]:
# from funcMidi import funcCreatNoteDic
from funcMidi import *
import copy
import itertools

# choNote = [41, 48, 52, 56, 60, 65]
# choNote = [41, 48, 53]
# choNote = [41, 48, 52, 56]
# choNote = [65, 61, 56, 46]
# choNote = [48, 56]
# choNote = [64, 60, 55, 52, 48]
# choNote = [64, 60, 55 ]
# choNote = [43, 47, 67, 50, 55, 59]

# choNote = [48, 52, 55, 60 ,64]
choNote = [42, 66, 58, 62]

dicNoteOnFingerBoard = funcCreatNoteDic()


chordSolution = funcChordSolution(choNote, dicNoteOnFingerBoard)


chordSolution

capo = 0
tuning = [64, 59, 55, 50, 45, 40]
Notes in Finger board:
[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]
[59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
[45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]


[[[6, 2, 1], [1, 2, 2], [3, 3, 3], [2, 3, 4]],
 [[6, 2, 1], [1, 2, 1], [3, 3, 3], [2, 3, 4]],
 [[6, 2, 1], [1, 2, 1], [3, 3, 2], [2, 3, 4]],
 [[6, 2, 1], [1, 2, 1], [3, 3, 2], [2, 3, 3]]]

In [1]:
def funcChordSolution(choNote, dicNoteOnFingerBoard):
    domains = funcNoteDomains(choNote, dicNoteOnFingerBoard)
    dicCombination = funcCSP(domains)
    dicCombPreproc, matchDict = funcAdversarykill(dicCombination)
    chordSolution = funcChordSol(dicCombPreproc, matchDict)
    return chordSolution

In [2]:
def funcNoteDomains(choNote, dicNoteOnFingerBoard):
    domains = []
    x = 0 # variable
    for x in range(len(choNote)):
        # x = 41
        domains.append([])
        fingerings = dicNoteOnFingerBoard[choNote[x]]
        # fingerings = [[5, 2], [6, 7]]
        for fingering in fingerings:
            # fingering = [5,2]
            if fingering[1] == 0: # if fret=0, no need swap all fingers
                domains[x].append(fingering + [0])
            else: # if fret !=0, swap all fingers 1~4
                for finger1234 in range (1,5):
    #                 print(finger1234)
                    domains[x].append(fingering + [finger1234])
    return domains

In [3]:
def dicAppend(key, value, dic):
#     useFlag[dn][pn] = 1
#     useFlag[sdn][spn] = 1
    if key in dic:
        dic[key].append(value)
    else:
        dic[key] = [value]
        
def funcCSP(domains):
    dicCombination = {} # "legal" combinations after DPC algorithm
    dn = 0 # domain number

    for dn in range(len(domains) -1): # domain number
        for pn in range(len(domains[dn])): # position number
            # sdn: sub-domain number, starting from the next domain
            for sdn in range(dn+1, len(domains)): 

                for spn in range(len(domains[sdn])):
                    # constraints

    #               # c0: see this position is already been justified as usable, then skip
                    string0 = domains[dn][pn][0]
                    string1 = domains[sdn][spn][0]
                    # c1: one note per string
                    if string0 != string1:
                        finger0 = domains[dn][pn][2]
                        finger1 = domains[sdn][spn][2]
                        fret0 = domains[dn][pn][1]
                        fret1 = domains[sdn][spn][1]
                        fingerDiff = finger0 - finger1 # finger difference
                        absFingerDiff = abs(fingerDiff)
                        fretDiff = fret0 - fret1
                        absFretDiff = abs(fretDiff)
                        
                        # fret0, no limitation
                        if fret0 == 0 or fret1 == 0:
                            dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                        # c4*: the same fret
                        elif fret0 == fret1:
                            #  c4: barre index
                            if finger0 == finger1 ==1:
                                dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                            # c4-1: same fret: lower finger press higher "strings"
                            elif (string0 - string1)*(finger0 - finger1) < 0:
                                dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)

                        # c2: higher finger press higher frets
                        #     finger0 is stronger
                        elif (fret0 - fret1) <0:
                            if finger0 == 1:
                                if finger1 == 2 and 0 < (-fretDiff) <= 2:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                                elif finger1 == 3 and 0 < (-fretDiff) <= 3:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                                elif finger1 == 4 and 0 < (-fretDiff) <= 4:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                            elif finger0 == 2:
                                if finger1 == 3 and 0 < (-fretDiff) <= 1:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                                elif finger1 == 4 and 0 < (-fretDiff) <= 3:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                            elif finger0 == 3:
                                if finger1 == 4 and 0 < (-fretDiff) <= 1:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                        elif (fret0 - fret1) >0: 
                        # when finger1 stronger than finger0
                            if finger1 == 1:
                                if finger0 == 2 and 0 < (fretDiff) <= 2:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                                elif finger0 == 3 and 0 < (fretDiff) <= 3:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                                elif finger0 == 4 and 0 < (fretDiff) <= 4:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                            elif finger1 == 2:
                                if finger0 == 3 and 0 < (fretDiff) <= 1:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                                elif finger0 == 4 and 0 < (fretDiff) <= 3:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
                            elif finger1 == 3:
                                if finger0 == 4 and 0 < (fretDiff) <= 1:
                                    dicAppend((dn,sdn),(domains[dn][pn], domains[sdn][spn]),dicCombination)
    return dicCombination



In [4]:
# Preprocessing: Adversary kill method. Non-lambda version. (Mao)
def funcAdversarykill(dicCombination):
    for loop in range(2):

        # 測試用. 目前做一次不乾淨，要做兩次
        if loop == 1:
            dicCombination = copy.deepcopy(result)


        Keys = dicCombination.keys()
        flatKey = sum(Keys, ())
        uniqueKey = set(flatKey)

        matchDict = {}
        for uniKey in uniqueKey:
#             print (uniKey)
            sets = []
            for key in Keys:
                if uniKey in key:
                    indexLoc = key.index(uniKey)
                    sets.append(set(map(tuple, list(zip(*dicCombination[key]))[indexLoc])))
#             print (set.intersection(*sets))
            matchDict[uniKey] = set.intersection(*sets)

        result = {}
        for k in dicCombination:
            result_temp = []
            compare1 = matchDict[k[0]]
    #         print("compare1 = " + str(compare1))
            compare2 = matchDict[k[1]]
            for item in dicCombination[k]:
    #             print("item = " + str(item))
    #             print("item[0] = " + str(item[0]))
    #             print("tuple(item[0]) = " + str(tuple(item[0])))
                if tuple(item[0]) in compare1 and tuple(item[1]) in compare2:
                    result_temp.append(item)
            result[k] = result_temp
    return result, matchDict

In [5]:
# cartisian product -- matchDict
def funcChordSol(dicCombPreproc, matchDict):
# NEED import itertools

    listMatchDict = []

    for key, value in matchDict.items():
        listMatchDict.append(list(value))

    listCartisianProduct = []
    for iterItem in itertools.product(*listMatchDict):
        listItem = []
        for tupleItem in iterItem:
            listItem.append(list(tupleItem))
        listCartisianProduct.append(listItem)

    def rmCandidate():                
        for i in range(lenSol):
            for j in range(i+1, lenSol):
                candPair = (candidate[i], candidate[j])
                if i ==1 and j == 2:
#                 if ( candPair ) not in dicCombPreproc[(i, j)]:
                    return False
        return True

    lenSol = len(listCartisianProduct[0])
    
    def judgeCandidateIn(candidate):  
        lenCand = len(candidate)
        for i in range(lenCand):
            for j in range(i+1, lenCand):
                candPair = (candidate[i], candidate[j])
                if ( candPair ) not in dicCombPreproc[(i, j)]:
                    return False
        return True

    def delFaultBarreIndex(candidate):
        oneCount = 0 # index finger counter
        indexFingerCount = 0
        fret = []
        indexFingerFret = []

        # 食指數目>1 and 食指的數目 < 食指fret的數目, 就代表有其他手指按了封閉的fret --> false
        for position in candidate:
            if position[2] == 1:
                indexFingerCount = indexFingerCount + 1
                indexFingerFret.append(position[1])

        uniqFret = list(set(indexFingerFret))
        if len(uniqFret) != 1:
            print("Index finger presses more than 1 fret. --> Error")
            return False
        elif len(uniqFret) ==1:
            uniqFret = uniqFret[0]

        fretCount = sum([1 for x in candidate if x[1]==uniqFret])
        if fretCount > indexFingerCount > 1:
            return False
        else:
            return True

    solutions = [x for x in listCartisianProduct if judgeCandidateIn(x)]    
    solutions = [x for x in solutions if delFaultBarreIndex(x)]
    return solutions


    # listMatchDict
    # listCartisianProduct
#     proposeSols